In [1]:
import os
import pandas as pd
import numpy as np
import requests
import datetime as dt

In [2]:
noaa_source_spreadsheets = 'https://www.glerl.noaa.gov/data/dashboard/data/hydroIO/sourceSpreadsheets/'

In [3]:
#ice data
path = os.path.join('..','data_files','ice_data')
erie = 'erieIceData.csv'
huron = 'huronIceData.csv'
mi = 'miIceData.csv'
ontario = 'ontarioIceData.csv'
superior = 'superiorIceData.csv'
files = [erie,huron,mi,ontario,superior]
df = pd.DataFrame()
sources = []

In [4]:
for e in files:
    new_df = pd.read_csv(os.path.join(path,e))
    lake = new_df.columns[0].replace(':','')
    sources.append({'Lake':lake,
                   'Sources':new_df.iloc[0]})
    new_df.columns = ['Date',f'{lake}']
    new_df = new_df.drop([0,1])
    if len(df) == 0:
        df = new_df
    else:
        df = df.merge(new_df,how='outer',left_on='Date',right_on ='Date')
df.head()

,Date,Lake Erie,Lake Huron,Lake Michigan,Lake Ontario,Lake Superior
0,12/20/1972,0,25.26,9.21,0.61,16.62
1,12/28/1972,0,12.65,12.09,2.98,9.35
2,01/02/1973,3.4,22.0,6.53,2.8,13.03
3,01/09/1973,25.4,31.3,21.55,21.4,29.69
4,01/16/1973,42.3,25.3,22.39,11.7,15.25


In [5]:
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by='Date').reset_index(drop=True)
df.dtypes

Date             datetime64[ns]
Lake Erie                object
Lake Huron               object
Lake Michigan            object
Lake Ontario             object
Lake Superior            object
dtype: object

In [6]:
df

,Date,Lake Erie,Lake Huron,Lake Michigan,Lake Ontario,Lake Superior
0,1972-12-19,NaN,NaN,NaN,NaN,0
1,1972-12-20,0,25.26,9.21,0.61,16.62
2,1972-12-28,0,12.65,12.09,2.98,9.35
3,1973-01-02,3.4,22.0,6.53,2.8,13.03
4,1973-01-09,25.4,31.3,21.55,21.4,29.69
...,...,...,...,...,...,...
2519,2016-05-01,0,0,0,0,0.55
2520,2016-05-02,0,0,0,0,0.24
2521,2016-05-03,0,0,0,0,0.24
2522,2016-05-04,0,0,0,0,0.24


In [7]:
max(df['Date'])

Timestamp('2016-05-05 00:00:00')

In [8]:
min(df['Date'])

Timestamp('1972-12-19 00:00:00')

In [9]:
df[pd.isnull(df).any(axis=1)]

,Date,Lake Erie,Lake Huron,Lake Michigan,Lake Ontario,Lake Superior
0,1972-12-19,NaN,NaN,NaN,NaN,0
17,1973-04-07,NaN,0.0,NaN,NaN,0
37,1974-05-03,NaN,0.0,0,NaN,0
56,1975-04-25,NaN,0.0,0,NaN,0
76,1976-04-21,0.0,0.0,0,NaN,0
...,...,...,...,...,...,...
1973,2013-11-23,NaN,NaN,NaN,NaN,0
2166,2014-06-06,NaN,NaN,NaN,NaN,0
2167,2014-11-13,NaN,NaN,NaN,NaN,0
2362,2015-05-29,NaN,NaN,NaN,NaN,0


In [10]:
df['Weekday'] = df['Date'].dt.day_name()
df['Date'] = df['Date'].dt.date
df['Weekday'].value_counts()

Monday       564
Thursday     441
Wednesday    422
Friday       419
Tuesday      291
Saturday     205
Sunday       182
Name: Weekday, dtype: int64

In [11]:
columns = df.columns.drop(['Date','Weekday'])
df[columns] = df[columns].apply(pd.to_numeric)

In [12]:
df.dtypes

Date              object
Lake Erie        float64
Lake Huron       float64
Lake Michigan    float64
Lake Ontario     float64
Lake Superior    float64
Weekday           object
dtype: object

In [14]:
df.max()

Date             2016-05-05
Lake Erie               100
Lake Huron             98.5
Lake Michigan         93.33
Lake Ontario          86.26
Lake Superior           100
Weekday           Wednesday
dtype: object

In [15]:
df[df['Lake Erie']==100]

,Date,Lake Erie,Lake Huron,Lake Michigan,Lake Ontario,Lake Superior,Weekday
108,1978-02-08,100.0,76.6,45.32,25.90,38.38,Wednesday
109,1978-02-15,100.0,77.8,35.33,59.40,51.18,Wednesday
110,1978-02-22,100.0,93.8,68.29,51.50,85.22,Wednesday
111,1978-03-01,100.0,95.8,61.36,48.40,92.49,Wednesday
112,1978-03-08,100.0,93.7,53.79,43.70,83.43,Wednesday
132,1979-02-12,100.0,95.3,61.27,61.12,90.38,Monday
822,1996-02-05,100.0,90.9,67.81,44.80,87.61,Monday
827,1996-02-16,100.0,90.5,38.98,45.30,82.64,Friday


In [16]:
df[df['Lake Superior']==100]

,Date,Lake Erie,Lake Huron,Lake Michigan,Lake Ontario,Lake Superior,Weekday
836,1996-03-08,88.4,98.2,61.62,6.9,100.0,Friday


In [17]:
diff_ls = [{'Date':df['Date'][0],'Days_since':np.nan,'Weekday':df['Weekday'][0]}]
diff_groups = []
for i in range(1,len(df)):
    diff = (df['Date'][i]-df['Date'][i-1]).days
    if diff > 100 or i==len(df)-1:
        diff_groups.append(diff_ls)
        diff_ls = [{'Date':df['Date'][i],'Days_since':diff,'Weekday':df['Weekday'][i]}]
    else:
        diff_ls.append({'Date':df['Date'][i],'Days_since':diff,'Weekday':df['Weekday'][i]})

In [18]:
len(diff_groups)

44

In [19]:
phases = []
for e in diff_groups:
    phases.append({'Start':e[0]['Date'],
                  'End':e[-1]['Date'],
                  'Entries':len(e),
                  'Days in Range':(e[-1]['Date']-e[0]['Date']).days})
    #print(f"Phase {diff_groups.index(e)}: {e[0]['Date']} to {e[-1]['Date']}, Entry count = {len(e)}")

In [20]:
phases_df = pd.DataFrame(phases)
phases_df['% Coverage'] = round((phases_df['Entries']/phases_df['Days in Range'])*100,2)
phases_df

,Start,End,Entries,Days in Range,% Coverage
0,1972-12-19,1973-04-07,18,109,16.51
1,1973-12-30,1974-05-03,20,124,16.13
2,1975-01-01,1975-04-25,19,114,16.67
3,1975-12-22,1976-04-21,20,121,16.53
4,1976-12-15,1977-05-05,23,141,16.31
5,1977-12-20,1978-05-04,22,135,16.30
6,1978-12-17,1979-05-15,25,149,16.78
7,1980-01-01,1980-04-17,18,107,16.82
8,1980-12-23,1981-04-16,18,114,15.79
9,1981-12-20,1982-05-18,24,149,16.11


In [23]:
phases_df.to_csv(os.path.join('..', 'data_files', 'prep_data','data_collection_phases.csv'))

In [27]:
start_date = phases_df.iloc[22]['Start']
end_date = phases_df.iloc[26]['End']

chosen_df_start = df[df['Date']>=start_date]
chosen_df = chosen_df_start[chosen_df_start['Date']<=end_date]
chosen_df.to_csv(os.path.join('..','data_files','clean_data','ice_coverage.csv'))